In [317]:
import pickle
with open('datasets/data.pkl', 'rb') as f:
    urls_frame = pickle.load(f)

with open('datasets/indices.pkl', 'rb') as f:
    indices = pickle.load(f)

In [335]:
url_array = np.zeros((100, 50000, 50+1))

for i, url in enumerate(urls_frame['url']):
    for j, c in enumerate(url):
        url_array[j, i, indices.get(c, 50)] = 1

MemoryError: 

In [258]:
import numpy as np

def url2tensor(url):
    res = torch.Tensor(np.zeros((len(url), 1, char_count+1)))
    for i, c in enumerate(url):
        index = indices.get(c, 100)
        res[i, 0, index] = 1
    return res

def row2vars(row):
    url = row[1][0]
    label = int(row[1][1] == 'good')
    in_tensor = url2tensor(url)
    out_tensor = torch.LongTensor(np.array([label]))
    return Variable(in_tensor), Variable(out_tensor)

N = 5
pairs = [row2vars(row) for row in list(urls_frame.iterrows())[:N]]

In [259]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(101, 50, 1)
        self.final = nn.Linear(50, 2)
    
    def forward(self, x):
        out, h = self.lstm(x)
        return self.final(out[-1])

In [273]:
# takes input urls, returns Pr{malicious}

model = Model()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(100):
    for X, y in pairs:
        pred = model(X)
    #     print(type(y.data))
    #     print(type(pred.data))
        loss = criterion(pred, y[0])
    #     print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print(loss)

Variable containing:
1.00000e-02 *
  1.7594
[torch.FloatTensor of size 1]



In [269]:
def check_accuracy(model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()
    for X, y in loader:
        X_var = Variable(X, volatile=True)

        scores = model(X_var)
        preds = scores.data.cpu() > 0.5

        num_correct += (preds == y).sum()
        num_samples += preds.size(0)

    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    

def train(model, loader_train, criterion, optimizer):
    model.train()
    for t, (X, y) in enumerate(loader_train):
        X_var = Variable(X)
        y_var = Variable(y).long()

        scores = model(X_var)

        loss = criterion(scores, y_var)
        if (t+1) % args.print_every == 0:
            print('t = %d, loss = %.4f' % (t+1, loss.data[0]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()